In [21]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [ ]:
!pip3 install tensorflow-gpu==1.15

In [5]:
# !pip install --upgrade spacy


In [5]:
import spacy
import spacy.cli
#spacy.cli.download("en_core_web_lg")

you don't have permission in Colab to load the model with normal spacy usage. 
Instead do the below steps

In [6]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [17]:
search_doc = nlp("The game was tense but it was a nail biting finish in cricket")
main_doc = nlp("The way Brazil played is fabulous. The scored each penalty and the freekick was a dream.")

main_doc_nouns = nlp(' '.join([str(t) for t in main_doc if t.pos_ in ['NOUN', 'PROPN']]))
search_doc_nouns = nlp(' '.join([str(t) for t in search_doc if t.pos_ in ['NOUN', 'PROPN']]))

print(main_doc_nouns)
print(search_doc_nouns)

In [23]:
# remove stop wrrds
search_doc_no_stop_words = nlp(' '.join([str(t) for t in search_doc_nouns if not t.is_stop]))
main_doc_no_stop_words = nlp(' '.join([str(t) for t in main_doc_nouns if not t.is_stop]))

print(search_doc_no_stop_words.similarity(main_doc_no_stop_words))

0.4793999618502406


Spacy's sentence embeddings are just the average of all word vector embeddings taken individually. So if you have a sentence with negating words like "good" and "bad" their vectors might cancel each other out resulting in not so good contextual embeddings. Anyway you should get rid of stop words

In [ ]:
# pip install tensorflow==1.15
#run this in CLI

In [7]:
!pip install bert-serving-server  # server
!pip install bert-serving-client  # client, independent of `bert-serving-server`

In [ ]:
# run this in CLI
# !bert-serving-start -model_dir /Users/vcroopana/bertmodels/multi_cased_L-12-H-768-A-12/ -num_worker=1 -cpu

In [26]:
from bert_serving.client import BertClient
bc = BertClient()
print('done')

done


In [48]:
embedding = bc.encode(["The game was tense but it was a nail biting finish in cricket"])
# check the shape of embedding, it should be 1x768
print(embedding.shape)

(1, 768)


In [ ]:
embedding1 = bc.encode(["The way Brazil played is fabulous. The scored each penalty and the freekick was a dream."])
# check the shape of embedding, it should be 1x768
print(embedding.shape)
print(embedding1)

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]


In [12]:
print(get_cosine_similarity(embedding, embedding1))

0.9310798


In [13]:
#1. dataset - first three are similar and second three are dissimilar 

text1=["Trying to test how the method is for identical text.",
       "I like that food.",
       "Ram is very nice..",
      "Pure malt whiskey.",
      "It is a ferocious dog and it barks whenever anybody uknown comes near the house.",
      "It is the family cow.",
      "My driving license is my identity in USA as everything is linked to it."]
text2=["Trying to test how the method is for identical text.",
       "That dish is exciting",
       "Is Ram very nice?",
       "Fresh orange juice.",
       "The painting in the art gallery is so fantastic !",
       "I have been driving this sports car for last ten years and I am so satisfied! ",
       "The president led us to war and we lost that war! "]


In [7]:
# Let us define a function that calculates BERT similarity given a pair of text 
def calculate_BERT_similarity(text1, text2):
  embedding1 = bc.encode([text1])
  embedding2 = bc.encode([text2])
  # use the function defined above
  BERT_s=get_cosine_similarity(embedding1, embedding2)
  return BERT_s


In [33]:
import numpy as np
def text_pair_similarity_run_bert(text1,text2):
#     print(text1)
#     print ("\n")
#     print(text2)
#     print ("\n")
    bert_sim = np.round(calculate_BERT_similarity(text1,text2),3)
#     print(bert_sim)
#     print("BERT Similarity : "+str(bert_sim))
#     print ("=====================================\n")
    return bert_sim


In [67]:
import pandas as pd
annotations_data = pd.read_csv('/Users/vcroopana/Downloads/summer2020/superbowl/ip/SB_ad_annotations.csv', index_col=0) 
annotations_data['Keywords'] = annotations_data['Brand Name']\
                                .str.cat(annotations_data['Ad Name'], sep=" ")\
                                .str.cat(annotations_data['Key Terms  Round 1'], sep=" ")
df = annotations_data.drop_duplicates()
print(df.shape)
annotations_data.head(2)

(75, 10)


,Brand Name,Ad Name,Product,Key Terms Round 1,KeyTerms_Edited,Excitatory Potential,Emotional vs. Rational,Semantic Affinity,Valence,Keywords
Ad Number,,,,,,,,,,
1,Trailer,Fast & Furious 9 Trailer,Movie Trailer,"fast and the furious, fast & the furious, fast...","fast_and_the_furious, fast_&_the_furious, fast...",1,1,2,1,Trailer Fast & Furious 9 Trailer fast and the...
2,Quibi,Quibi Bank Heist,Video Platform,"quibi, bank heist, robbery, less than ten minu...","quibi, bank_heist, robbery, less_than_ten_minu...",2,1,2,2,"Quibi Quibi Bank Heist quibi, bank heist, rob..."


In [68]:
man_ann_data = pd.read_csv(r'/Users/vcroopana/Downloads/summer2020/superbowl/mann_ann_sb.csv')    
man_ann_data.head(1)

,Ò,user_id,tweet_id,time_of_tweet,user_location,team followed,affective_state,tweet_text,ad_keywords,ad_mentioned,ad_manual
0,64628,2.836102e+09,1.220000e+18,Mon Feb 03 03:53:56 +0000 2020,NaN,both,1,"Man I wanna look at the brightside ""we thought...","John legend, Chrissy Teigen, genesis, hyundai,...",genesis going away party,none


In [69]:
from nltk.corpus import stopwords
import re

stop = stopwords.words('english')
print(len(stop))

def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac    
    text = textBeforeMention+ " " + textAfterMention  
    return text

def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    
    # remove punctuations except '_'
    punctuation = ['(', ')', '[',']','?', ':', ':', ',', '.', '!', '/', '"', "'", '@', '#', '&']
#     text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 
    text = "".join((char for char in text if char not in punctuation))
    
#     text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K james_bond") )

179
cadillacabc joinrt 31k james_bond


In [70]:

ad_keywords_clean = annotations_data['Keywords'].apply(lambda ad: cleanTweet(ad))
man_ann_data['tweet_clean'] = man_ann_data['tweet_text'].apply(lambda twt: cleanTweet(twt))
embedding_tweets = man_ann_data['tweet_clean'].apply(lambda x: bc.encode([x]))
print("Embeddings calculated")
bert_sim_df = pd.DataFrame(columns = annotations_data['Ad Name'])


Embeddings calculated


In [88]:
ad_id = 1
for ad in ad_keywords_clean:
    embedding_ad = bc.encode([ad])
    bert_sim_col =[]
    print("computin sim for ad:"+ str(ad_id))
    for i in range(0, len(embedding_tweets)): 
        bert_sim = get_cosine_similarity(embedding_ad, embedding_tweets[i])
        bert_sim_col.append(np.round(bert_sim,3))
    
    bert_sim_df[annotations_data['Ad Name'][ad_id]] = bert_sim_col
    ad_id = ad_id + 1
bert_sim_df.head(3)
print("Bert Similarities Calculated")

Bert Similarities Calculated


In [87]:
import numpy as np

nlargest = 5
order = np.argsort(-bert_sim_df.values, axis=1)[:, :nlargest]
result = pd.DataFrame(bert_sim_df.columns[order], 
                      columns=['top{}_ad'.format(i) for i in range(1, nlargest+1)],
                      index= bert_sim_df.index)

order_vals = np.sort(-bert_sim_df.values, axis=1)[:, :nlargest]
result_vals = pd.DataFrame(-order_vals,
                          columns = ['top{}'.format(i) for i in range(1, nlargest+1)],
                          index= bert_sim_df.index)

result_con = pd.concat([result_vals, result], axis =1)
man_ann_data_merged = pd.concat([man_ann_data, result_con], axis =1)

man_ann_data_merged.head()

print("Top n similar ads computed")

Top n similar ads computed


/Users/vcroopana/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Support for multi-dimensional indexing (e.g. `index[:, None]`) on an Index is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  """


In [11]:
def get_conf_matrix(ad_manual, ad_algo, ad_algo_2, ad_prob, ad_prob2, thresh_prob):
    res = ""
    ad_manual = ad_manual.lower()
    ad_algo = ad_algo.lower()
    ad_algo_2 = ad_algo_2.lower()
    if(ad_manual == ad_algo and ad_prob >= thresh_prob):
        res = 'TP'
    elif(ad_manual == ad_algo and ad_prob < thresh_prob):
        res = 'FN'
    elif(ad_manual=='none' and ad_prob > thresh_prob):
        res = 'FP'
    elif(ad_manual=='none' and ad_prob < thresh_prob):
        res = 'TN'
    elif(ad_manual!='none' and ad_manual!= ad_algo and ad_prob >= thresh_prob):
        res = 'FP'
    elif(ad_manual!='none' and ad_manual!= ad_algo and ad_prob < thresh_prob):
        res = 'FN'
    return res
def get_conf_matrix_2(ad_manual, ad_algo, ad_algo_2, ad_algo_3, ad_algo_4, ad_algo_5):
    res = ""
    ad_manual = ad_manual.lower()
    ad_algo = ad_algo.lower()
    ad_algo_2 = ad_algo_2.lower()
    ad_algo_3 = ad_algo_3.lower()
    ad_algo_4 = ad_algo_4.lower()
    ad_algo_5 = ad_algo_5.lower()
    
    if(ad_manual == ad_algo or ad_manual == ad_algo_2 or ad_manual == ad_algo_3 or ad_manual == ad_algo_4
      or ad_manual == ad_algo_5):
        res = 'TP'
    elif(ad_manual=='none'):
        res = 'FP'
    elif(ad_manual!='none' and ad_manual!= ad_algo and ad_manual!= ad_algo_2 and ad_manual!= ad_algo_3
        and ad_manual!= ad_algo_4 and ad_manual!= ad_algo_5):
        res = 'FN'
    elif(ad_manual!='none' and ad_manual!= ad_algo and ad_manual!= ad_algo_2 and ad_manual!= ad_algo_3
        and ad_manual!= ad_algo_4 and ad_manual!= ad_algo_5):
        res = 'TN'
    return res

def computeAccuracy(result):    
    n_tp = result[result['conf_matrix'] == 'TP'].shape[0]
    n_fp = result[result['conf_matrix'] == 'FP'].shape[0]
    n_fn = result[result['conf_matrix'] == 'FN'].shape[0]
    n_tn = result[result['conf_matrix'] == 'TN'].shape[0]
    
    print("n_tp:" + str(n_tp)+ " n_fp:" + str(n_fp)+ " n_fn:" + str(n_fn) + " n_tn:" + str(n_tn))

    precision = n_tp/(n_tp+ n_fp)
    recall = n_tp/(n_tp+ n_fn)
    f_measure = (2*precision*recall)/ (precision+recall)

    return precision, recall, f_measure

man_ann_data_merged['conf_matrix'] = man_ann_data_merged.apply(lambda x: get_conf_matrix(x['ad_manual'], x['top1_ad'], 
                                                          x['top2_ad'], x['top1'], x['top2'], 0.8), axis =1)
# man_ann_data_merged['conf_matrix'] = man_ann_data_merged.apply(lambda x: get_conf_matrix_2(x['ad_manual'], x['top1_ad'], 
#                             x['top2_ad'], x['top3_ad'], x['top4_ad'], x['top5_ad']), axis =1)
computeAccuracy(man_ann_data_merged)

n_tp:103 n_fp:2396 n_fn:0 n_tn:1


(0.041216486594637856, 1.0, 0.07916986933128363)

In [12]:
man_ann_data_merged.to_csv("/Users/vcroopana/Downloads/summer2020/superbowl/sim_bert_0.8.csv")

In [4]:
import pandas as pd
man_ann_data_merged = pd.read_csv("/Users/vcroopana/Downloads/summer2020/superbowl/sim_bert.csv")

In [ ]:
for i in range(0,7):
    text_pair_similarity_run_bert(text1[i],text2[i])    

In [ ]:
harphampeg lynnelanae wgravelee darngoood franklin_graham pepsi idea feel female given
proctor gamble procter gamble come together procter gamble come together sofia vergara chili bounty old spice head shoulders olay charming febreze
